In [ ]:
!pip install praw

In [ ]:
import praw
from datetime import datetime, timedelta
import pandas as pd
from tqdm import tqdm
import ast

In [ ]:
# import praw
# from datetime import datetime, timedelta

# # Replace these with your own credentials
# client_id = 'RYzupE7lyg9j-eiAZt1fXA'
# client_secret = 'yofGU7bS7r7Y7-o8KGGf5YwsFkSI8g'
# user_agent = 'Alive-Influence-2987'

# # Initialize the Reddit API client
# reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)

# keyword1 = 'Israel'  # Replace with your desired keyword
# keyword2 = 'war'  # Replace with your desired country
# keywords = ["Israel", "Palestine", "conflict","war","United States"]
# print(" OR ".join(keywords))

# # Set the time threshold (10 hours ago)
# time_threshold = int((datetime.utcnow() - timedelta(hours=100)).timestamp())
# print(time_threshold)
# # Initialize variables to keep track of the highest upvoted post
# highest_upvotes = 0
# best_post = None

# print(len(list(reddit.subreddit("all").search(" AND ".join(keywords), limit=None, sort="top"))))

# # Loop through the search results for all subreddits
# for submission in reddit.subreddit("all").search(" AND ".join(keywords), limit=None, sort="top"):
# #     print(submission.created_utc)
#     if submission.created_utc < time_threshold:
        
#         # Stop searching if we've gone back in time more than 10 hours
#         continue
#     print(submission.created_utc)
#     # Check if the post was created in the specific country
#     print('\n\nPost:\n',submission.title)
#     print(submission.__dict__)
#     if submission.score > highest_upvotes:
#         highest_upvotes = submission.score
#         best_post = submission

# if best_post:
#     print(f'Most upvoted post in the last 10 hours from {keyword2} containing "{keyword1}":')
#     print(f'Title: {best_post.title}')
#     print(f'URL: {best_post.url}')
#     print(f'Upvotes: {best_post.score}')
#     print(best_post.__dict__)
# else:
#     print(f'No matching posts found in the last 10 hours from {keyword2} containing "{keyword1}"')

In [ ]:
keywords_df=pd.read_csv('/kaggle/input/final-project/keywords.csv')

In [ ]:
keywords_df.head()

In [ ]:
keywords_df['flag']=keywords_df['flag'].fillna(1)

In [ ]:
keywords_df.head()

In [ ]:
keywords=list(keywords_df[keywords_df['flag']==1]['keywords'])

In [ ]:
keywords

In [ ]:


# Replace these with your own credentials
client_id = 'RYzupE7lyg9j-eiAZt1fXA'
client_secret = 'yofGU7bS7r7Y7-o8KGGf5YwsFkSI8g'
user_agent = 'Alive-Influence-2987'

# Initialize the Reddit API client
reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)
# Define the subreddit and keywords
subreddit_name = "canada"
# keywords = ['hostages', 'million', 'survivors', 'fighting', 'guterres', 'palestinians',  'war',   'israelpalestine' , 'military',  'hamas', 'humanitarian', 'supplies', 'palestinian', 'israeli',  'unrwa',  'israel', 'line', 'gaza',  'death', 'strip']

# Define the number of posts to retrieve
num_posts = 20
num_comments=50

# Create an empty list to store data
posts_data = []
delta = datetime.utcnow() - timedelta(days=60)

# Retrieve posts from the subreddit
subreddit = reddit.subreddit(subreddit_name)
num_posts_scrapable=len(list(subreddit.search(" OR ".join(keywords), limit=num_posts)))
print(num_posts_scrapable)
for submission in tqdm(subreddit.search(" OR ".join(keywords), limit=num_posts), total=num_posts_scrapable, desc="Scraping Posts"):
#     print('Scraping')
    if datetime.utcfromtimestamp(submission.created_utc) >= delta:
        post_data = {
            "title": submission.title,
            "body": submission.selftext,
            "url": submission.url,
            "score": submission.score,
            "created_utc": submission.created_utc,
            "comments": []  # Initialize an empty list for comments
        }

        submission.comments.replace_more(limit=num_comments)
        for comment in submission.comments.list():
#             if isinstance(comment, praw.models.MoreComments):
#                 continue  # Skip MoreComments objects

            comment_data = {
                "comment_body": comment.body,
                "comment_score": comment.score,
                "comment_created_utc": comment.created_utc,
            }
            post_data["comments"].append(comment_data)

        posts_data.append(post_data)
    
    if len(posts_data)==50:
        break

# Create a DataFrame from the collected data
df = pd.DataFrame(posts_data)

# Set the post title as the index if required
# df.set_index("title", inplace=True)


In [ ]:
df

In [ ]:
for t in range(len(df['title'])):
    print(str(t)+' : '+df['title'][t])

In [ ]:
10,14

In [ ]:
# comments=[]
for c in df.iloc[16:17,5]:
    for com in c:
        if len(comments)==300:
            break
        comments.append(com['comment_body'])

In [ ]:
len(comments)

In [ ]:
comments

In [ ]:
df.shape

In [ ]:
row_to_drop=[3,38]

In [ ]:
for x in range(len(df['title'])):
    print(str(x)+' : '+df['title'][x])

In [ ]:
df.to_excel('cananda.xlsx',index=False)

In [ ]:
rows_to_drop=[3,38]

In [ ]:
rows_to_drop

In [ ]:
df_dropped=df.drop(rows_to_drop)

In [ ]:
# df_droppped=pd.read_excel('/kaggle/input/final-project/AskIndia_dropped.xlsx')

In [ ]:
df_dropped=df

In [ ]:
len(df_dropped['comments'])

In [ ]:
comments=[]
for c in df_dropped['comments']:
    print(len(c))
    for comment in c:
        comments.append(comment['comment_body'])
#     print(c[1]['comment_body'])

In [ ]:
len(comments)

In [ ]:
comments[:10]

In [ ]:
comments=pd.DataFrame({'comments':comments})

In [ ]:
comments

In [ ]:
comments.to_csv('cananda_manual_raw.csv',index=False)